In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\yadav\\chatbot\\End-to-End-medical-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\yadav\\chatbot\\End-to-End-medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


c:\Users\yadav\.conda\envs\farmerchat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Extract Data from PDF Files
def load_pdf_file(data) :
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    print(f'You have {len(documents)} documents in your data')
    return documents

In [7]:
%pwd


'c:\\Users\\yadav\\chatbot\\End-to-End-medical-chatbot'

In [8]:
import os
os.listdir()

['.env',
 '.git',
 '.gitignore',
 'app.py',
 'Data',
 'Generative_AI_Project.egg-info',
 'LICENSE',
 'README.md',
 'requirements.txt',
 'research',
 'setup.py',
 'src',
 'store_index.py',
 'template.py',
 'templates']

In [9]:
extracted_data = load_pdf_file(data="Data/")

You have 460 documents in your data


In [10]:
# extracted_data

In [11]:
#split the data into text chunks

def text_split(extracted_data) :
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    print(f'Now you have {len(text_chunks)} chunks of data')
    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)

Now you have 2157 chunks of data


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the embedding model from HuggingFace

def download_hugging_face_embeddings() :
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\yadav\AppData\Local\Temp\ipykernel_13660\135544804.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
query_result = embeddings.embed_query("What is COVID-19?")
print("length",len(query_result)) 

length 384


In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")


In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "agriculture-chatbot"
pc.create_index(name=index_name,
                dimension=384, 
                metric="cosine", 
                spec=ServerlessSpec(
                    cloud="aws", region="us-east-1")
        )

{
    "name": "agriculture-chatbot",
    "metric": "cosine",
    "host": "agriculture-chatbot-kywhcku.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [24]:
#Embed each chunk and upsert the embeddings to Pinecone
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

c:\Users\yadav\.conda\envs\farmerchat\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [25]:
#Load existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(embedding=embeddings,index_name=index_name)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("What is COVID-19?")

In [29]:
retrieved_docs

[Document(id='e4ff2703-2c3d-49c8-ae2c-f600d16fd9f0', metadata={'author': 'DR.RAGHU', 'creationdate': '2020-01-07T12:58:43+05:30', 'creator': 'Acrobat PDFMaker 8.1 for Word', 'moddate': '2020-01-07T13:00:12+05:30', 'page': 296.0, 'page_label': '297', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'Data\\AGRICULTURE.pdf', 'title': '', 'total_pages': 460.0}, page_content='CoC 25 10-12 145.7 12.77 18.62 \nCoG 6 10-12 140.6 13.34 18.39'),
 Document(id='ee3efec8-bead-43c8-b4f1-f090bd57f6fc', metadata={'author': 'DR.RAGHU', 'creationdate': '2020-01-07T12:58:43+05:30', 'creator': 'Acrobat PDFMaker 8.1 for Word', 'moddate': '2020-01-07T13:00:12+05:30', 'page': 206.0, 'page_label': '207', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'Data\\AGRICULTURE.pdf', 'title': '', 'total_pages': 460.0}, page_content='nercrosis virus)  \n(Vector: Thrips tabaci, \nFrankliniella schultzeii) \n \n• Antiviral principles (AVP) from sorghum or coconut leaves. \nAVPs are extracted as foll

In [30]:
GEMENIE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [31]:
os.environ["GEMENIE_API_KEY"] = GEMENIE_API_KEY

In [32]:
!pip install langchain-google-genai


In [33]:
!pip show langchain-google-genai
!pip list | grep langchain



Name: langchain-google-genai
Version: 2.1.12
Summary: An integration package connecting Google's genai package and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Users\yadav\.conda\envs\farmerchat\Lib\site-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Required-by: 


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [34]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",   # or "gemini-1.5-pro"
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=os.getenv("GEMENIE_API_KEY")  # ✅ make sure the env var is set correctly
)


In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Updated Prompt to match the required Gemini format
prompt_template = (
    "Context: {context}\n\n"
    "Input: {input}\n\n"
    "Provide me the response from the context according to the input from the user. "
    "The response should be formal and clear, maintaining the wording style of the user's input. "
    "Ensure the response is strictly from the context, declarative, and follows the user's tone. "
    "Modify the given context accordingly."
)

# Chat Prompt (Ensuring Correct Query Structure for Gemini)
prompt = ChatPromptTemplate.from_messages(
    [
        (prompt_template),
    ]
)


In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [38]:
response = rag_chain.invoke({"input": "what is crop?"})
print(response["answer"])

Based on the provided text, crops include perennial crops, pure border crops, mixed crops, and intercrops.  Examples of crops mentioned are green gram, black gram, soybean, castor, sorghum, and cotton.
